# Very naive learning on the vector of surrounding bases with xgboost

---

### Analysis

I will try to naively predict the mehtylation value (0,1) based on naive sequence context feauture vectors.

I will use xgboost, a sophisticated library for gradient boosting, which is used to win many kaggle competitions.
    



### Conlcusions

- suprisingly, with brutal overfitting generalization still improves


---

In [1]:
import numpy as np
import time
import xgboost as xgb

import sys
sys.path.append('../my_modules/')
from loading_utils import read_my_data,create_sets

import os,subprocess
workdir='/nagyvinyok/adat84/sotejedlik/ribli/methylation_code/modelling'
subprocess.call(['mkdir',workdir])
os.chdir(workdir)

### Load data

In [2]:
x,y= read_my_data(fname='../prepare_data/meth_feat_vect.csv')

#select the whole dataset
(train_x,train_y),(valid_x,valid_y),(test_x,test_y)= create_sets(
    x,y,N_train=50000,N_valid=5000,N_test=5000,length=1000)

Loading data... 


#### Create xgboost matrices

In [3]:
dtrain = xgb.DMatrix( train_x, label=train_y)
dvalid = xgb.DMatrix( valid_x, label=valid_y)
dtest = xgb.DMatrix( test_x, label=test_y)

### With modest overfitting

In [4]:
param = {'max_depth':4,
         'eta':0.1,
         'min_child_weight':50,
         'colsample_bytree':0.8,
         'subsample':0.6,
         'silent':1,
         'objective': "binary:logistic",
         'eval_metric': 'error',
         'nthread':8}

evallist  = [(dtrain,'train'),(dvalid,'eval')]

#train
num_round = 500
bst = xgb.train(param,
                dtrain,
                evals=evallist,
                num_boost_round=num_round,
                early_stopping_rounds=100,
                verbose_eval=50)

Will train until eval error hasn't decreased in 100 rounds.
[0]	train-error:0.415020	eval-error:0.415200
[50]	train-error:0.245100	eval-error:0.277000
[100]	train-error:0.219760	eval-error:0.248000
[150]	train-error:0.207740	eval-error:0.244400
[200]	train-error:0.199100	eval-error:0.241000
[250]	train-error:0.191360	eval-error:0.238400
[300]	train-error:0.182300	eval-error:0.235400
[350]	train-error:0.173160	eval-error:0.232200
[400]	train-error:0.165160	eval-error:0.230200
[450]	train-error:0.156560	eval-error:0.231600
Stopping. Best iteration:
[395]	train-error:0.165940	eval-error:0.229000



#### Final scores

In [5]:
print 'train score:',list(map(round,bst.predict(dtrain))==train_y).count(True)/float(len(train_y))
print 'validation score:',list(map(round,bst.predict(dvalid))==valid_y).count(True)/float(len(valid_y))
print 'test score:',list(map(round,bst.predict(dtest))==test_y).count(True)/float(len(test_y))

train score: 0.85056
validation score: 0.7682
test score: 0.7646


### With brutal overfitting

In [7]:
param = {'max_depth':7,
         'eta':0.01,
         'min_child_weight':1,
         'colsample_bytree':1,
         'subsample':1,
         'silent':1,
         'objective': "binary:logistic",
         'eval_metric': 'error',
         'nthread':8}

evallist  = [(dtrain,'train'),(dvalid,'eval')]

#train
num_round = 5000
bst = xgb.train(param,
                dtrain,
                evals=evallist,
                num_boost_round=num_round,
                early_stopping_rounds=5000,
                verbose_eval=500)

Will train until eval error hasn't decreased in 5000 rounds.
[0]	train-error:0.388680	eval-error:0.417600
[500]	train-error:0.150880	eval-error:0.235800
[1000]	train-error:0.111840	eval-error:0.227200
[1500]	train-error:0.072180	eval-error:0.226400
[2000]	train-error:0.037960	eval-error:0.223400
[2500]	train-error:0.016060	eval-error:0.220000
[3000]	train-error:0.007140	eval-error:0.220400
[3500]	train-error:0.002600	eval-error:0.218400
[4000]	train-error:0.001140	eval-error:0.218000
[4500]	train-error:0.000280	eval-error:0.218400
[4999]	train-error:0.000120	eval-error:0.216000


In [8]:
print 'train score:',list(map(round,bst.predict(dtrain))==train_y).count(True)/float(len(train_y))
print 'validation score:',list(map(round,bst.predict(dvalid))==valid_y).count(True)/float(len(valid_y))
print 'test score:',list(map(round,bst.predict(dtest))==test_y).count(True)/float(len(test_y))

train score: 0.99988
validation score: 0.784
test score: 0.7834
